In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium as folium
import geopandas as gpd
import json
import os
import sys
import branca.colormap as cm
import matplotlib
import plotly.express as px
import plotly.graph_objects as go

%load_ext lab_black

In [ ]:
pop_prov_df = pd.read_excel("./data/pobmun/pobmun22.xlsx", header=1)

In [ ]:
pop_prov_df

In [ ]:
provincias_geojson = "./data/georef-spain-provincia@public.geojson"

In [ ]:
with open("./data/georef-spain-provincia@public.geojson", "r") as f:
    province_data = json.load(f)

## Renaming so that Geojson and DF match names

In [ ]:
prov_names = []
for i in range(len(province_data["features"])):
    prov_names.append(province_data["features"][i]["properties"]["prov_name"])

In [ ]:
province_rename_dict = {
    "Alicante/Alacant": "Alacant",
    "Araba/Álava": "Araba",
    "Balears, Illes": "Illes Balears",
    "Castellón/Castelló": "Castelló",
    "Coruña, A": "A Coruña",
    "Palmas, Las": "Las Palmas",
    "Rioja, La": "La Rioja",
    "Valencia/València": "València",
}

In [ ]:
# Replace with my province dict
pop_prov_df["PROVINCIA"] = pop_prov_df["PROVINCIA"].replace(province_rename_dict)

In [ ]:
# Check if all provinces are in the geojson
for i in pop_prov_df["PROVINCIA"].unique():
    if i not in prov_names:
        print(i)

In [ ]:
male_pop = pop_prov_df.groupby("PROVINCIA")["HOMBRES"].sum()
female_pop = pop_prov_df.groupby("PROVINCIA")["MUJERES"].sum()

pop_df = pd.DataFrame({"men": male_pop, "women": female_pop})
pop_df = pop_df.sort_values(by="men", ascending=False)

In [ ]:
plt.figure(figsize=(10, 10))
pop_df.plot(kind="barh", figsize=(10, 10), width=0.8, cmap="tab20c")
plt.title("Population by province")
plt.xlabel("Population")
plt.ylabel("Province")
plt.show()

In [ ]:
pop_df["total"] = pop_df["men"] + pop_df["women"]

In [ ]:
pop_df.reset_index(inplace=True)

In [ ]:
# Add 'total' to GeoJSON properties
for feature in province_data["features"]:
    prov_name = feature["properties"]["prov_name"]
    if prov_name == "Territorio no asociado a ninguna provincia":
        # make it 0
        feature["properties"]["total"] = 0
        continue
    total_population = pop_df.loc[pop_df["PROVINCIA"] == prov_name, "total"].values[0]
    feature["properties"]["total"] = int(total_population)

In [ ]:
# cm.linear

In [ ]:
# Get min and max population
min_pop = pop_df["total"].min()
max_pop = pop_df["total"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_pop, max_pop)

In [ ]:
f = folium.Figure(width=1200, height=1000)

m = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f)

folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "total"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population",
).add_to(m)


# add labels
style_function = lambda feature: {
    "fillColor": colormap(feature["properties"]["total"]),
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.50,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "total"],
    aliases=["Province", "Population"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="Provinces",
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

## Log Scale pop

In [ ]:
# Use logarithmic scale for the total population to enhance the differences
pop_df["log_total"] = np.log(pop_df["total"])

In [ ]:
missing_prov = []

for code in prov_names:
    if code not in pop_prov_df["PROVINCIA"].unique():
        missing_prov.append(code)

province_data["features"] = [
    feature
    for feature in province_data["features"]
    if feature["properties"]["prov_name"] not in missing_prov
]

In [ ]:
# Add 'total' to GeoJSON properties
for feature in province_data["features"]:
    prov_name = feature["properties"]["prov_name"]
    try:
        total_population = pop_df.loc[
            pop_df["PROVINCIA"] == prov_name, "log_total"
        ].values[0]
        feature["properties"]["log_total"] = int(total_population)
    except Exception as e:
        print(e)
        print(prov_name)

In [ ]:
# Do it but with log scale

min_log_pop = pop_df["log_total"].min()
max_log_pop = pop_df["log_total"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_log_pop, max_log_pop)

In [ ]:
f = folium.Figure(width=1200, height=1000)

m = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f)

folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "log_total"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population",
).add_to(m)


# add labels
style_function = lambda feature: {
    "fillColor": colormap(np.log(feature["properties"]["total"])),
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.50,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "log_total"],
    aliases=["Province", "Population"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="Provinces",
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
pop_df = pop_df.assign(m_w_ratio=pop_df["men"] / pop_df["women"])
pop_df.head()

In [ ]:
# Add 'm_w_ratio' to GeoJSON properties
for feature in province_data["features"]:
    prov_name = feature["properties"]["prov_name"]
    if prov_name == "Territorio no asociado a ninguna provincia":
        # make it 0
        feature["properties"]["m_w_ratio"] = 1
        continue
    m_w_ratio = pop_df.loc[pop_df["PROVINCIA"] == prov_name, "m_w_ratio"].values[0]
    feature["properties"]["m_w_ratio"] = float(m_w_ratio)

In [ ]:
# Get min and max population
min_ratio = pop_df["m_w_ratio"].min()
max_ratio = pop_df["m_w_ratio"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_ratio, max_ratio)
# colormap = cm.linear.YlGn_09.scale(min_pop, max_pop)

In [ ]:
f2 = folium.Figure(width=1200, height=1000)

m2 = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f2)
folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "m_w_ratio"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="men/women ratio",
).add_to(m2)

# add labels
# Create style function using the color function
style_function = lambda feature: {
    "fillColor": colormap(feature["properties"]["m_w_ratio"]),
    "color": "#000000",
    "fillOpacity": 0,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "m_w_ratio"],
    aliases=["Province", "Men per Woman Ratio"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="m_w_ratio",
).add_to(m2)

folium.LayerControl().add_to(m2)

In [ ]:
m2

In [ ]:
# Provinces with most women:
print("Provinces with most women:")
print(
    pop_df[["PROVINCIA", "m_w_ratio"]]
    .sort_values(by="m_w_ratio")
    .head(10)
    .to_markdown()
)
pop_df[["PROVINCIA", "m_w_ratio"]].sort_values(by="m_w_ratio").head(10).plot(
    kind="bar", x="PROVINCIA", ylim=[0.9, 0.98], cmap="tab20c"
)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("Provinces with most women")
plt.xlabel("Province")
plt.ylabel("Man/Woman Ratio")
# no legend
plt.legend().remove()
plt.show()

In [ ]:
print("Provinces with most men:")
print(
    pop_df[["PROVINCIA", "m_w_ratio"]]
    .sort_values(by="m_w_ratio", ascending=False)
    .head(10)
    .to_markdown()
)
pop_df[["PROVINCIA", "m_w_ratio"]].sort_values(by="m_w_ratio", ascending=False).head(
    10
).plot(kind="bar", x="PROVINCIA", ylim=[0.98, 1.06], cmap="tab20c")
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("Provinces with most men")
plt.xlabel("Province")
plt.ylabel("Man/Woman Ratio")
# no legend
plt.legend().remove()
plt.show()

In [ ]:
# Plot a bar chart that is the nr of provinces with m_w_ratio larger than 1 vs smaller than 1
pop_df["m_w_ratio"].apply(lambda x: 1 if x > 1 else 0).value_counts().plot(
    kind="bar", x="m_w_ratio"
)
plt.title("Count of provinces with Man/Woman > 1 vs < 1")
plt.xticks([0, 1], ["More women", " More men"], rotation=0)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.xlabel("Amount of men per woman")
plt.ylabel("count")
plt.bar_label(plt.gca().containers[0], fmt="%d")
plt.show()

In [ ]:
# Bar plot of absolut values of men vs women
pop_df[["men", "women"]].sum().div(1000000).plot(kind="bar")
plt.title("Total population in Spain")
plt.xticks(rotation=0)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.ylabel("Population (millions)")
plt.bar_label(ax.containers[0], fmt="%.1f")
plt.show()

# Mortality

In [ ]:
pop_df["PROVINCIA"].unique()

In [ ]:
mortality_raw = pd.read_csv("./data/mortalidad.csv", sep=";")

In [ ]:
mortality_raw.head()

In [ ]:
mortality_raw["Provincias"].unique()

In [ ]:
mortality_raw["Funciones"].unique()

In [ ]:
mortality_raw[
    (mortality_raw["Funciones"] == "Esperanza de vida")
    & (mortality_raw["Periodo"] == 2021)
]

In [ ]:
life_exp = mortality_raw[mortality_raw["Funciones"] == "Esperanza de vida"].copy()

In [ ]:
life_exp["Total"] = life_exp["Total"].str.replace(",", ".").astype(float)

In [ ]:
for edad in life_exp["Edad"].unique():
    _temp_mort = life_exp[
        (life_exp["Edad"] == edad)
        & (life_exp["Periodo"] == 2021)
        & (life_exp["Sexo"] == "Ambos sexos")
    ]
    _temp_mort.set_index("Provincias", inplace=True)
    _temp_mort["Total"].sort_values(ascending=False).plot(
        kind="bar", x="Provincias", figsize=(10, 5)
    )
    plt.title(f"Life expectancy at {edad} ")
    plt.show()

## Rebuild this in plotly with dropdown for age selection

In [ ]:
mortality_raw[
    (mortality_raw["Funciones"] == "Supervivientes")
    & (mortality_raw["Periodo"] == 2021)
]

## Nombres Más frecuentes


In [ ]:
most_common_names_men = pd.read_excel(
    "./data/nombres_mas_frecuentes.xls", sheet_name="ESPAÑA_100_hombres", header=4
)
most_common_names_women = pd.read_excel(
    "./data/nombres_mas_frecuentes.xls", sheet_name="ESPAÑA_100_mujeres", header=4
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 15))
sns.barplot(
    data=most_common_names_men,
    x="FRECUENCIA",
    y="NOMBRE COMPLETO",
    ax=ax[0],
)
sns.barplot(
    data=most_common_names_women,
    x="FRECUENCIA",
    y="NOMBRE COMPLETO",
    ax=ax[1],
)
ax[0].tick_params(axis="y", labelsize=8)
ax[1].tick_params(axis="y", labelsize=8)
ax[0].set_title("Men")
ax[1].set_title("Women")
ax[0].set_ylabel("Name")
ax[1].set_ylabel("")  # Remove y label from second subplot
# common y label

plt.suptitle("100 Most frequent names 01/01/2022", y=0.92)
plt.show()

## Nombres por edad media

In [ ]:
cols = ["orden", "nombre", "frecuencia", "edad_media"]
names_age_men = pd.read_excel("./data/nombres_por_edad_media.xls", header=6, names=cols)
names_age_women = pd.read_excel(
    "./data/nombres_por_edad_media.xls", header=6, names=cols, sheet_name="Mujeres"
)

In [ ]:
names_age_men

In [ ]:
top_100_men = names_age_men.head(100).sort_values(by="edad_media")
top_100_women = names_age_women.head(100).sort_values(by="edad_media")

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))
sns.barplot(
    data=top_100_men,
    x="edad_media",
    y="nombre",
)
# plot bar labels
for i, v in enumerate(top_100_men["edad_media"]):
    ax.annotate(str(v), xy=(v, i), ha="left", va="center")

# make fontsize smaller
ax.tick_params(axis="y", labelsize=8)

plt.title("Average age for the 100 most frequent names")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))
sns.barplot(
    data=top_100_women,
    x="edad_media",
    y="nombre",
)
# plot bar labels
for i, v in enumerate(top_100_women["edad_media"]):
    ax.annotate(str(v), xy=(v, i), ha="left", va="center")

# make fontsize smaller
ax.tick_params(axis="y", labelsize=8)

plt.title("Average age for the 100 most frequent names")
plt.show()

In [ ]:
names_age_men["source"] = "men"
names_age_women["source"] = "women"

all_names = pd.concat((names_age_men, names_age_women))

In [ ]:
all_names

In [ ]:
all_names["age_group"] = pd.cut(
    all_names["edad_media"],
    [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 85, 90],
    labels=[
        "0-5",
        "5-10",
        "10-15",
        "15-20",
        "20-25",
        "25-30",
        "30-35",
        "35-40",
        "40-45",
        "45-50",
        "50-55",
        "55-60",
        "60-65",
        "65-70",
        "70-75",
        "75-85",
        "85-90",
    ],
)

In [ ]:
# for each age group and sex get the top 3 names
top_3_names = (
    all_names.groupby(["source", "age_group"])
    .apply(lambda x: x.nlargest(3, "frecuencia"))
    .reset_index(drop=True)
)

In [ ]:
top_3_names[top_3_names["age_group"] == "75-85"]

## The lower part is wrong. This is not the top name per age group.
Its the group with average and highest freq

In [ ]:
for group in top_3_names["age_group"].unique():
    _temp = top_3_names[top_3_names["age_group"] == group]
    fig, ax = plt.subplots(1, figsize=(10, 10))
    sns.barplot(
        data=_temp,
        x="frecuencia",
        y="nombre",
        hue="source",
        # palette=["#1f77b4", "#ff7f0e"],
    )
    # plot bar labels
    for i, v in enumerate(_temp["frecuencia"]):
        ax.annotate(str(v), xy=(v, i), ha="left", va="center")

    plt.title(f"Top 3 names for age group {group}")
    plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

for group in top_3_names["age_group"].unique():
    _temp = top_3_names[top_3_names["age_group"] == group]

    # Prepare data for wordcloud: a dictionary where keys are names and values are frequencies
    data = _temp.set_index("nombre")["frecuencia"].to_dict()

    # Generate word cloud
    wordcloud = WordCloud(width=1000, height=500).generate_from_frequencies(data)

    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Top 3 names for age group {group}")
    plt.show()

In [ ]:
import matplotlib as mpl